разбиваем наших клиентов на сегментов по RFM

In [112]:
!pip install clickhouse_driver
!pip install h3

In [113]:
import pandas as pd
import numpy as np
import datetime
import sqlalchemy
import seaborn as sns
from clickhouse_driver import Client
from pandas_profiling import ProfileReport
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import tools
from plotly.subplots import make_subplots
import plotly.express as px
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans
import h3
import seaborn as sns
import matplotlib.pyplot as plt

In [114]:
client = Client('89.208.84.253',
                port= '59000',
                user='student',
                password='nUrHdn2N',
                secure=False,
                verify=True,
                database='data_analysis',
                compression=False)

In [115]:
result,columns = client.execute("""SELECT v.idhash_client,
       max(order_dttm) as max_dttm_order,
       count(idhash_order)as count_order,
       sum(v.client_bill_usd) as sum_order
FROM data_analysis.views v inner join orders o on v.idhash_order = o.idhash_order
GROUP BY v.idhash_client
ORDER BY v.idhash_client ASC;""",with_column_types=True)

In [116]:
orders_RFM = pd.DataFrame(result, columns=[tuple[0] for tuple in columns])

In [117]:
orders_RFM.head()

,idhash_client,max_dttm_order,count_order,sum_order
0,39262,2015-12-19 22:42:41,5,9.55
1,48902,2015-12-14 12:32:11,1,2.65
2,199395,2015-12-17 10:12:34,4,19.11
3,420016,2015-12-19 22:40:34,1,4.08
4,1721255,2015-12-17 23:41:47,3,11.96


In [118]:
a = datetime.datetime(2016, 1, 1, 0, 0, 0)
print(a)

2016-01-01 00:00:00


In [119]:
orders_RFM.insert(2, "Difference", (a - orders_RFM['max_dttm_order']).dt.days , True)

In [120]:
orders_RFM.head()

,idhash_client,max_dttm_order,Difference,count_order,sum_order
0,39262,2015-12-19 22:42:41,12,5,9.55
1,48902,2015-12-14 12:32:11,17,1,2.65
2,199395,2015-12-17 10:12:34,14,4,19.11
3,420016,2015-12-19 22:40:34,12,1,4.08
4,1721255,2015-12-17 23:41:47,14,3,11.96


In [121]:
a1 = np.percentile(orders_RFM.Difference, 33)
a1

15.0

In [122]:
a2 = np.percentile(orders_RFM.Difference, 66)
a2

20.0

In [123]:
orders_RFM.insert(3, "R", np.where(orders_RFM['Difference'] <= a1,3,np.where(orders_RFM['Difference'] <= a2,2,1)), True)

In [124]:
orders_RFM.head()

,idhash_client,max_dttm_order,Difference,R,count_order,sum_order
0,39262,2015-12-19 22:42:41,12,3,5,9.55
1,48902,2015-12-14 12:32:11,17,2,1,2.65
2,199395,2015-12-17 10:12:34,14,3,4,19.11
3,420016,2015-12-19 22:40:34,12,3,1,4.08
4,1721255,2015-12-17 23:41:47,14,3,3,11.96


In [125]:
b1 = np.percentile(orders_RFM.count_order, 33)
b1

1.0

In [126]:
b2 = np.percentile(orders_RFM.count_order, 66)
b2

3.0

In [127]:
orders_RFM.insert(5, "F", np.where(orders_RFM['count_order'] <= b1 ,1,np.where(orders_RFM['count_order'] <= b2,2,3)), True)

In [128]:
orders_RFM.head()

,idhash_client,max_dttm_order,Difference,R,count_order,F,sum_order
0,39262,2015-12-19 22:42:41,12,3,5,3,9.55
1,48902,2015-12-14 12:32:11,17,2,1,1,2.65
2,199395,2015-12-17 10:12:34,14,3,4,3,19.11
3,420016,2015-12-19 22:40:34,12,3,1,1,4.08
4,1721255,2015-12-17 23:41:47,14,3,3,2,11.96


In [129]:
c1 = np.percentile(orders_RFM.sum_order, 33)
c1

5.65

In [130]:
c2 = np.percentile(orders_RFM.sum_order, 66)
c2

14.18

In [131]:
orders_RFM.insert(7, "M", np.where(orders_RFM['sum_order'] <= c1 ,1,np.where(orders_RFM['sum_order'] <= c2,2,3)), True)

In [132]:
orders_RFM.head()

,idhash_client,max_dttm_order,Difference,R,count_order,F,sum_order,M
0,39262,2015-12-19 22:42:41,12,3,5,3,9.55,2
1,48902,2015-12-14 12:32:11,17,2,1,1,2.65,1
2,199395,2015-12-17 10:12:34,14,3,4,3,19.11,3
3,420016,2015-12-19 22:40:34,12,3,1,1,4.08,1
4,1721255,2015-12-17 23:41:47,14,3,3,2,11.96,2


In [133]:
orders_RFM.insert(8, "RFM", orders_RFM['R']*100 + orders_RFM['F']*10 + orders_RFM['M'], True)

In [134]:
orders_RFM.head()

,idhash_client,max_dttm_order,Difference,R,count_order,F,sum_order,M,RFM
0,39262,2015-12-19 22:42:41,12,3,5,3,9.55,2,332
1,48902,2015-12-14 12:32:11,17,2,1,1,2.65,1,211
2,199395,2015-12-17 10:12:34,14,3,4,3,19.11,3,333
3,420016,2015-12-19 22:40:34,12,3,1,1,4.08,1,311
4,1721255,2015-12-17 23:41:47,14,3,3,2,11.96,2,322


разбиваем наших клиентов на сегменты по RFM

In [135]:
orders_RFM[['idhash_client', 'RFM']].head()

,idhash_client,RFM
0,39262,332
1,48902,211
2,199395,333
3,420016,311
4,1721255,322


Для удобства я разбила данные всех клиентов на 27 сегментов на 3 группы.

In [136]:
orders_RFM.insert(9, "S", np.where(orders_RFM['R'] == 1 ,'Потерянные',np.where(orders_RFM['R'] == 2,'Спящие','Постоянные')), True)

In [137]:
orders_RFM.head()

,idhash_client,max_dttm_order,Difference,R,count_order,F,sum_order,M,RFM,S
0,39262,2015-12-19 22:42:41,12,3,5,3,9.55,2,332,Постоянные
1,48902,2015-12-14 12:32:11,17,2,1,1,2.65,1,211,Спящие
2,199395,2015-12-17 10:12:34,14,3,4,3,19.11,3,333,Постоянные
3,420016,2015-12-19 22:40:34,12,3,1,1,4.08,1,311,Постоянные
4,1721255,2015-12-17 23:41:47,14,3,3,2,11.96,2,322,Постоянные


In [157]:
client_RFM = orders_RFM.filter(['idhash_client', 'S'], axis=1)

In [159]:
client_RFM.head(14)

,idhash_client,S
0,39262,Постоянные
1,48902,Спящие
2,199395,Постоянные
3,420016,Постоянные
4,1721255,Постоянные
5,2082469,Потерянные
6,2468535,Потерянные
7,2825748,Потерянные
8,3322714,Постоянные
9,4098821,Потерянные


Задания на метрики

In [141]:
result,columns = client.execute("""SELECT v.idhash_client,
       toDate(o.order_dttm) as order_data,
       countIf(o.idhash_order, o.idhash_order > 0) as count_order
FROM data_analysis.orders o inner join data_analysis.views v ON o.idhash_order = v.idhash_order
GROUP BY v.idhash_client, order_data
ORDER BY v.idhash_client ASC;""",with_column_types=True)
metrix_order = pd.DataFrame(result, columns=[tuple[0] for tuple in columns])

In [142]:
result,columns = client.execute("""SELECT idhash_client,
       toDate(view_dttm) as view_data,
       count(idhash_view) as count_view
FROM data_analysis.views
GROUP BY idhash_client, toDate(view_dttm)
ORDER BY idhash_client ASC;""",with_column_types=True)
metrix_view = pd.DataFrame(result, columns=[tuple[0] for tuple in columns])

In [143]:
data1 = client_RFM.join(metrix_view.set_index('idhash_client'), on='idhash_client',how = 'inner')

In [144]:
data1[['idhash_client', 'RFM', 'view_data','count_view']].head()

,idhash_client,RFM,view_data,count_view
0,39262,332,2015-12-19,7
0,39262,332,2015-12-15,7
0,39262,332,2015-12-17,1
0,39262,332,2015-12-12,7
0,39262,332,2015-12-08,3


In [145]:
view_RFM = data1.drop(['idhash_client'],axis = 1)

In [146]:
view_RFM.head()

,RFM,view_data,count_view
0,332,2015-12-19,7
0,332,2015-12-15,7
0,332,2015-12-17,1
0,332,2015-12-12,7
0,332,2015-12-08,3


In [147]:
m = view_RFM.groupby(['RFM','view_data'], as_index=False).count_view.sum()

In [148]:
m

,RFM,view_data,count_view
0,111,2015-12-06,2092
1,111,2015-12-07,2120
2,111,2015-12-08,3318
3,111,2015-12-09,3411
4,111,2015-12-10,2506
...,...,...,...
338,333,2015-12-14,11970
339,333,2015-12-15,12772
340,333,2015-12-16,15041
341,333,2015-12-17,16068


In [149]:
data2 = client_RFM.join(metrix_order.set_index('idhash_client'), on='idhash_client',how = 'inner')
data2[['idhash_client', 'RFM', 'order_data','count_order']].head()

,idhash_client,RFM,order_data,count_order
0,39262,332,2015-12-19,1
0,39262,332,2015-12-15,1
0,39262,332,2015-12-12,2
0,39262,332,2015-12-08,1
1,48902,211,2015-12-14,1


In [150]:
order_RFM = data2.drop(['idhash_client'],axis = 1)
order_RFM.head()

,RFM,order_data,count_order
0,332,2015-12-19,1
0,332,2015-12-15,1
0,332,2015-12-12,2
0,332,2015-12-08,1
1,211,2015-12-14,1


In [151]:
m1 = order_RFM.groupby(['RFM','order_data'], as_index=False).count_order.sum()

In [152]:
m1

,RFM,order_data,count_order
0,111,2015-12-06,379
1,111,2015-12-07,423
2,111,2015-12-08,661
3,111,2015-12-09,664
4,111,2015-12-10,556
...,...,...,...
204,333,2015-12-15,3159
205,333,2015-12-16,3841
206,333,2015-12-17,3912
207,333,2015-12-18,1


In [153]:
metrix.head()

,idhash_client,order_data,count_order
0,39262,2015-12-19,1
1,39262,2015-12-15,1
2,39262,2015-12-12,2
3,39262,2015-12-08,1
4,48902,2015-12-14,1


In [154]:
metrix1.head()

,idhash_client,view_data,count_view
0,39262,2015-12-19,7
1,39262,2015-12-15,7
2,39262,2015-12-17,1
3,39262,2015-12-12,7
4,39262,2015-12-08,3


In [155]:
data.head()

,idhash_client,view_data,count_view,order_data,count_order
0,39262,2015-12-19,7,2015-12-19,1.0
0,39262,2015-12-19,7,2015-12-15,1.0
0,39262,2015-12-19,7,2015-12-12,2.0
0,39262,2015-12-19,7,2015-12-08,1.0
1,39262,2015-12-15,7,2015-12-19,1.0


In [156]:
metrix.groupby(lambda x: metrix['view_dttm'][x].day).head()

KeyError: ignored

In [ ]:
metrix.head()

In [ ]:
def View2Order(metrix):
    for i in (metrix['view_dttm'].dt.day):
        metrix['count_order']/metrix['count_view']
        

In [ ]:
View2Order(metrix)